In [0]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("YourAppName") \
    .getOrCreate()

In [0]:
spark.conf.set("spark.databricks.io.cache.enabled", "false")
print("spark.databricks.io.cache.enabled is %s" % spark.conf.get("spark.databricks.io.cache.enabled"))

spark.databricks.io.cache.enabled is false


In [0]:
%pip install pandas

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import pandas as pd
import numpy as np
#import databricks.koalas as ks
# import dask.dataframe as dd
# from dask.distributed import Client, LocalCluster

print('pandas version: %s' % pd.__version__)

print('numpy version: %s' % np.__version__)

#print('koalas version: %s' % ks.__version__)

# import dask
# print('dask version: %s' % dask.__version__)

import pyarrow
print('pyarrow version: %s' % pyarrow.__version__)

import pyspark
print('pyspark version: %s' % pyspark.__version__)


import time

def benchmark(f, df, benchmarks, name, **kwargs):
    """Benchmark the given function against the given DataFrame.
    
    Parameters
    ----------
    f: function to benchmark
    df: data frame
    benchmarks: container for benchmark results
    name: task name
    
    Returns
    -------
    Duration (in seconds) of the given operation
    """
    start_time = time.time()
    ret = f(df, **kwargs)
    benchmarks['duration'].append(time.time() - start_time)
    benchmarks['task'].append(name)
    print(f"{name} took: {benchmarks['duration'][-1]} seconds")
    return benchmarks['duration'][-1]

def get_results(benchmarks):
    """Return a pandas DataFrame containing benchmark results."""
    return pd.DataFrame.from_dict(benchmarks)

pandas version: 1.4.2
numpy version: 1.21.5
pyarrow version: 7.0.0
pyspark version: 3.3.2.dev0


In [0]:
file_path = "/FileStore/tables/yellow_trip_5_months.parquet"
#pandas_data = pd.read_parquet(file_path, engine="pyarrow")
pandas_data = spark.read.format('parquet').options(header='true').load(file_path).toPandas()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1433832980640685>:3
      1 file_path = "/FileStore/tables/yellow_trip_5_months.parquet"
      2 #pandas_data = pd.read_parquet(file_path, engine="pyarrow")
----> 3 pandas_data = spark.read.format('parquet').options(header='true').load(file_path).toPandas()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:302, in DataFrameReader.load(self, path, format, schema, **options)
    300 self.options(**options)
    301 if isinstance(path, str):
--> 302     

In [0]:
pandas_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [0]:
pandas_benchmarks = {
    'duration': [],  # in seconds
    'task': [],
}

In [0]:
def read_file_parquet():
    return pd.read_parquet("/FileStore/tables/yellow_tripdata_2023_01.parquet")

def count(df):
    return len(df)

def count_index_length(df):
    return len(df.index)

def mean(df):
    return df['fare_amount'].mean()

def standard_deviation(df):
    return df['fare_amount'].std()

def mean_of_sum(df):
    return (df['fare_amount'] + df['tip_amount']).mean()

def sum_columns(df):
    return (df['fare_amount'] + df['tip_amount'])

def mean_of_product(df):
    return (df['fare_amount'] * df['tip_amount']).mean()

def product_columns(df):
    return (df['fare_amount'] * df['tip_amount'])

def value_counts(df):
    return df['fare_amount'].value_counts()

def mean_of_complicated_arithmetic_operation(df):
    theta_1 = df['start_lon']
    phi_1 = df['start_lat']
    theta_2 = df['end_lon']
    phi_2 = df['end_lat']
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2 +
            np.cos(theta_1*np.pi/180) * np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return ret.mean()

def complicated_arithmetic_operation(df):
    theta_1 = df['start_lon']
    phi_1 = df['start_lat']
    theta_2 = df['end_lon']
    phi_2 = df['end_lat']
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2 +
            np.cos(theta_1*np.pi/180) * np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return ret

def groupby_statistics(df):
    result = df.groupby('passenger_count').agg(
        fare_amount_mean=('fare_amount', 'mean'),
        fare_amount_std=('fare_amount', 'std'),
        tip_amount_mean=('tip_amount', 'mean'),
        tip_amount_std=('tip_amount', 'std')
    )
    return result

other = groupby_statistics(pandas_data)
other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])

def join_count(df, other):
    return len(pd.merge(df, other, left_index=True, right_index=True))

def join_data(df, other):
    return pd.merge(df, other, left_index=True, right_index=True)

In [0]:
other

,f_a,f_a,t_i,t_i
passenger_count,,,,
0.0,16.258301,16.109847,2.870892,3.319242
1.0,17.864871,17.208074,3.324993,3.748582
2.0,20.187780,20.151504,3.589881,4.222252
3.0,19.665432,20.457761,3.370632,4.091855
4.0,20.900318,22.510652,3.281927,4.365429
5.0,17.897049,15.882436,3.383753,3.623471
6.0,18.010731,16.029976,3.355087,3.670653
7.0,68.166667,11.788412,9.470000,8.057379
8.0,82.113077,2.666813,13.389231,6.765661


In [0]:
#benchmark(read_file_parquet, df=None, benchmarks=dask_benchmarks, name='read file')
benchmark(count, df=pandas_data, benchmarks=pandas_benchmarks, name='count')
benchmark(count_index_length, df=pandas_data, benchmarks=pandas_benchmarks, name='count index length')
benchmark(mean, df=pandas_data, benchmarks=pandas_benchmarks, name='mean')
benchmark(standard_deviation, df=pandas_data, benchmarks=pandas_benchmarks, name='standard deviation')
benchmark(mean_of_sum, df=pandas_data, benchmarks=pandas_benchmarks, name='mean of columns addition')
benchmark(sum_columns, df=pandas_data, benchmarks=pandas_benchmarks, name='addition of columns')
benchmark(mean_of_product, df=pandas_data, benchmarks=pandas_benchmarks, name='mean of columns multiplication')
benchmark(product_columns, df=pandas_data, benchmarks=pandas_benchmarks, name='multiplication of columns')
benchmark(value_counts, df=pandas_data, benchmarks=pandas_benchmarks, name='value counts')
# No column for this
# benchmark(mean_of_complicated_arithmetic_operation, df=dask_data, benchmarks=dask_benchmarks, name='mean of complex arithmetic ops')
# benchmark(complicated_arithmetic_operation, df=dask_data, benchmarks=dask_benchmarks, name='complex arithmetic ops')
benchmark(groupby_statistics, df=pandas_data, benchmarks=pandas_benchmarks, name='groupby statistics')
benchmark(join_count, pandas_data, benchmarks=pandas_benchmarks, name='join count', other=other)
benchmark(join_data, pandas_data, benchmarks=pandas_benchmarks, name='join', other=other) # cant join

count took: 0.00014901161193847656 seconds
count index length took: 7.152557373046875e-06 seconds
mean took: 0.015315055847167969 seconds
standard deviation took: 0.055608510971069336 seconds
mean of columns addition took: 0.026159286499023438 seconds
addition of columns took: 0.008296489715576172 seconds
mean of columns multiplication took: 0.01734137535095215 seconds
multiplication of columns took: 0.008443117141723633 seconds
value counts took: 0.08369731903076172 seconds
groupby statistics took: 0.22147178649902344 seconds
join count took: 0.7389745712280273 seconds
join took: 0.7035892009735107 seconds
Out[7]: 0.7035892009735107

Operations with filtering

In [0]:
expr_filter = (pandas_data.tip_amount >= 1) & (pandas_data.tip_amount <= 5)

def filter_data(df):
    return df[expr_filter]
  
pandas_filtered = filter_data(pandas_data)

In [0]:
benchmark(count, pandas_filtered, benchmarks=pandas_benchmarks, name='filtered count')
benchmark(count_index_length, pandas_filtered, benchmarks=pandas_benchmarks, name='filtered count index length')
benchmark(mean, pandas_filtered, benchmarks=pandas_benchmarks, name='filtered mean')
benchmark(standard_deviation, pandas_filtered, benchmarks=pandas_benchmarks, name='filtered standard deviation')
benchmark(mean_of_sum, pandas_filtered, benchmarks=pandas_benchmarks, name ='filtered mean of columns addition')
benchmark(sum_columns, df=pandas_filtered, benchmarks=pandas_benchmarks, name='filtered addition of columns')
benchmark(mean_of_product, pandas_filtered, benchmarks=pandas_benchmarks, name ='filtered mean of columns multiplication')
benchmark(product_columns, df=pandas_filtered, benchmarks=pandas_benchmarks, name='filtered multiplication of columns')
#benchmark(mean_of_complicated_arithmetic_operation, dask_filtered, benchmarks=dask_benchmarks, name='filtered mean of complex arithmetic ops')
#benchmark(complicated_arithmetic_operation, dask_filtered, benchmarks=dask_benchmarks, name='filtered complex arithmetic ops')
benchmark(value_counts, pandas_filtered, benchmarks=pandas_benchmarks, name ='filtered value counts')
benchmark(groupby_statistics, pandas_filtered, benchmarks=pandas_benchmarks, name='filtered groupby statistics')

other = groupby_statistics(pandas_filtered)
other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])

benchmark(join_count, pandas_filtered, benchmarks=pandas_benchmarks, name='filtered join count', other=other)
benchmark(join_data, pandas_filtered, benchmarks=pandas_benchmarks, name='filtered join', other=other)

filtered count took: 1.239776611328125e-05 seconds
filtered count index length took: 4.291534423828125e-06 seconds
filtered mean took: 0.0063076019287109375 seconds
filtered standard deviation took: 0.03155779838562012 seconds
filtered mean of columns addition took: 0.013622522354125977 seconds
filtered addition of columns took: 0.0085906982421875 seconds
filtered mean of columns multiplication took: 0.016338825225830078 seconds
filtered multiplication of columns took: 0.008226871490478516 seconds
filtered value counts took: 0.07968497276306152 seconds
filtered groupby statistics took: 0.11608219146728516 seconds
filtered join count took: 0.7741854190826416 seconds
filtered join took: 0.5692503452301025 seconds
Out[9]: 0.5692503452301025

In [0]:
pandas_res_temp = get_results(pandas_benchmarks).set_index('task')
pandas_res_temp

,duration
task,
count,0.000149
count index length,0.000007
mean,0.015315
standard deviation,0.055609
mean of columns addition,0.026159
addition of columns,0.008296
mean of columns multiplication,0.017341
multiplication of columns,0.008443
value counts,0.083697
